In [ ]:
#######################################################################
##  Made by: Dr. Keungoui Kim
##  Title: Scitech data import from Dimensions
##  goal : 
##  Data set:  
##  Time Span:
##  Variables
##      Input: 
##      Output: 
##  Time-stamp: #  "Sun Jan 26 01:47:34 2020":  edited by awe kim ; code
##  Notice :
#######################################################################

In [ ]:
### Library
import pandas as pd
import numpy as np
import math

### Directory Setting for Export
#dir = "E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/" # Home
dir = "D:/Google Drive(awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/" # HGU

In [ ]:
### API log in
import dimcli
dimcli.login(key="792DDFAFCCA7478D8F37159F274A2783",
             endpoint="https://app.dimensions.ai/api/dsl/v2")

dsl = dimcli.Dsl()

In [ ]:
# Extract Grant data
# Extract Funding data 
# What funding initiates science -> technology
# 
